starter code in but there is still an issue with incompatability with tensor flow see piazza for solution to issue

**Chapter 16 – Natural Language Processing with RNNs and Attention**

_This notebook contains all the sample code in chapter 16._

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jflanigan/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

TensorFlow 2.x selected.
     |████████████████████████████████| 1.0MB 9.6MB/s 
ERROR: tensorflow-federated 0.11.0 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.11.0 has requirement attrs~=18.2, but you'll have attrs 19.3.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement cachetools~=3.1.1, but you'll have cachetools 4.0.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement grpcio~=1.24.3, but you'll have grpcio 1.27.1 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement tensorflow~=2.0.0, but you'll have tensorflow 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement tensorflow-addons~=0.6.0, but you'll have tensorflow-addons 0.8.2 which is incompatible.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


# Sentiment Analysis

In [0]:
tf.random.set_seed(42)

You can load the IMDB dataset easily:

In [5]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()
#this in unimportant

17465344/17464789 [==============================] - 0s 0us/step


In [6]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [7]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1646592/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [8]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAF3B3I/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAF3B3I/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAF3B3I/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
#this does the split of 20000 train 5000 dev 25000 test
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'train[80%:]', 'test'),
    as_supervised=True)

In [10]:
train_examples_batch, train_labels_batch = next(iter(validation_data.batch(10)))
train_examples_batch
#train_labels_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"Instead, go to the zoo, buy some peanuts and feed 'em to the monkeys. Monkeys are funny. People with amnesia who don't say much, just sit there with vacant eyes are not all that funny.<br /><br />Black comedy? There isn't a black person in it, and there isn't one funny thing in it either.<br /><br />Walmart buys these things up somehow and puts them on their dollar rack. It's labeled Unrated. I think they took out the topless scene. They may have taken out other stuff too, who knows? All we know is that whatever they took out, isn't there any more.<br /><br />The acting seemed OK to me. There's a lot of unfathomables tho. It's supposed to be a city? It's supposed to be a big lake? If it's so hot in the church people are fanning themselves, why are they all wearing coats?",
       b'Well, was Morgan Freeman any more unusual as God than George Burns? This film sure was better than that bore, "Oh, God". I was totally engrossed and LMA

In [11]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [0]:
#this does size of data sets from imdb 
train_size = 20000 #something wrong with this hard coded till find right function
test_size = info.splits["test"].num_examples
validation_size = 5000 #hard coded until I can find the right function


In [13]:
train_size, test_size, validation_size

(20000, 25000, 5000)

In [14]:
for X_batch, y_batch in train_data.batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [0]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [16]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

the sections below handle the vocab for the model

In [0]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in train_data.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [18]:
vocabulary.most_common()[:3]

[(b'<pad>', 171811), (b'the', 48816), (b'a', 30677)]

In [19]:
len(vocabulary)

48238

In [0]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [21]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [0]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [23]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

This preproccesses the data batches and does batch size 

In [0]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch
#validation set and test set are pre proccessed now as well as the training set
#128 64 32 16 8 values for changing batch size
train_set = train_data.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)
validation_set = validation_data.batch(32).map(preprocess)
validation_set = validation_set.map(encode_words).prefetch(1)
test_set = test_data.batch(32).map(preprocess)
test_set = test_set.map(encode_words).prefetch(1)

In [0]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

In [83]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)


Epoch 1/5
625/625 [==============================] - 28s 45ms/step - loss: 0.5497 - accuracy: 0.7102
Epoch 2/5
625/625 [==============================] - 23s 36ms/step - loss: 0.3487 - accuracy: 0.8547
Epoch 3/5
625/625 [==============================] - 23s 36ms/step - loss: 0.1835 - accuracy: 0.9347
Epoch 4/5
625/625 [==============================] - 23s 36ms/step - loss: 0.1365 - accuracy: 0.9516
Epoch 5/5
625/625 [==============================] - 23s 37ms/step - loss: 0.0990 - accuracy: 0.9658


In [84]:
#testing example
model.evaluate(validation_set)

    157/Unknown - 4s 24ms/step - loss: 0.7355 - accuracy: 0.7658

[0.7355379895040184, 0.7658]

My implementation

In [0]:
#below is simple RNN implementation
# tests were done switching out values 
#based off test code above

In [0]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.SimpleRNN(128, activation ="relu", dropout=0.6),
    keras.layers.Dense(1, activation="sigmoid")
])
#model.add(keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                          # mask_zero=True, # not shown in the book
                           #input_shape=[None])) # embed to vectors
#model.add(keras.layers.SimpleRNN(128, return_state= True)) # get vector from them running through simplernn
#model.add(keras.layers.Dense(1, activation= "sigmoid")) # label at end
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)


In [66]:
#model.evaluate(train_set)
model.evaluate(validation_set)
#model.evaluate(test_set)

    157/Unknown - 3s 17ms/step - loss: 1.3086 - accuracy: 0.5212

[1.3085722429737163, 0.5212]

In [81]:
#part 2 implementation
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.LSTM(128, dropout=.2), #return_sequences=True
    keras.layers.Dense(1, activation="sigmoid")
])
#model.add(keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                          # mask_zero=True, # not shown in the book
                           #input_shape=[None])) # embed to vectors
#model.add(keras.layers.SimpleRNN(128, return_state= True)) # get vector from them running through simplernn
#model.add(keras.layers.Dense(1, activation= "sigmoid")) # label at end
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
history = model.fit(train_set, epochs=10)
#eval against val set
model.evaluate(validation_set)

Epoch 1/10
625/625 [==============================] - 18s 29ms/step - loss: 0.5413 - accuracy: 0.7271
Epoch 2/10
625/625 [==============================] - 14s 22ms/step - loss: 0.4030 - accuracy: 0.8240
Epoch 3/10
625/625 [==============================] - 14s 23ms/step - loss: 0.3484 - accuracy: 0.8523
Epoch 4/10
625/625 [==============================] - 14s 23ms/step - loss: 0.3044 - accuracy: 0.8752
Epoch 5/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2629 - accuracy: 0.8936
Epoch 6/10
625/625 [==============================] - 14s 22ms/step - loss: 0.2203 - accuracy: 0.9154
Epoch 7/10
625/625 [==============================] - 14s 22ms/step - loss: 0.1799 - accuracy: 0.9329
Epoch 8/10
625/625 [==============================] - 14s 23ms/step - loss: 0.1464 - accuracy: 0.9482
Epoch 9/10
625/625 [==============================] - 14s 23ms/step - loss: 0.1155 - accuracy: 0.9594
Epoch 10/10
625/625 [==============================] - 14s 22ms/step - loss: 0.089

[0.8524988274665395, 0.73]

In [80]:
#model.evaluate(train_set)
#model.evaluate(validation_set)
model.evaluate(test_set)

    782/Unknown - 9s 11ms/step - loss: 1.1784 - accuracy: 0.7099

[1.1784067584959137, 0.70992]

In [0]:
ep = 3
while ep < 11:
  print ("next train set", ep)
  model = keras.models.Sequential([
      keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                            mask_zero=True, # not shown in the book
                            input_shape=[None]),
      keras.layers.LSTM(128, dropout=.2),
      keras.layers.Dense(1, activation="sigmoid")
  ])
  #model.add(keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                            # mask_zero=True, # not shown in the book
                            #input_shape=[None])) # embed to vectors
  #model.add(keras.layers.SimpleRNN(128, return_state= True)) # get vector from them running through simplernn
  #model.add(keras.layers.Dense(1, activation= "sigmoid")) # label at end
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  history = model.fit(train_set, epochs=ep)
  ep += 1
  model.evaluate(validation_set)
  print ("")

##############

Or using manual masking:

In [0]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

## Reusing Pretrained Embeddings

In [0]:
tf.random.set_seed(42)

In [0]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [0]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [0]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

In [0]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)